# Preamble

In [1]:
%%time

import snowflake.snowpark.modin.plugin
import modin.pandas as pd
import numpy as np
import datetime
import pandas as native_pd
from snowflake.snowpark.session import Session; session = Session.builder.create()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://snowbiz.okta.com/app/snowflake/exk8wfsfryJIn4IWZ2p7/sso/saml?SAMLRequest=jVJdc9owEPwrHvUZy3agBA2QoYE0bkgA25CZvAlbEBVZUnVyDP31lfnopA%2FJ9E1z2r3du73%2Bzb4U3hszwJUcoNAPkMdkrgoutwO0zO5a18gDS2VBhZJsgA4M0M2wD7QUmowq%2ByoT9qtiYD3XSAJpPgaoMpIoChyIpCUDYnOSjh6nJPIDQgGYsU4OnSkFcKf1aq0mGNd17ddXvjJbHAVBgIMedqgG8gW9k9Cfa2ijrMqVuFD2bqYPJEIctBsJh3AK8zPxG5enFXymsj6BgNxn2bw1n6UZ8kaX6W6VhKpkJmXmjedsmUxPBsA5SJ9mz%2FezZTrxQap6I%2BiO5arUlXXdfPfCG1Zgobbc7SgeD5De8WL0NS3MpBc%2BxjUcJt8XUfbQo%2BuHadLrLpJ0tV3p9TTRifjZWeTIW10SjZpEY4CKxbLJ0bpSEHVaQbsVhFkUkahLrtp%2BJ4xekDd2OXJJ7ZF5MdtYXPPfvtpZejRHtcZ%2FfWO2313XG9iYw49YtuPnl0h3MYDCTUzodCnkaMAM%2F3f%2BPn7POh%2Fbk9t%2FPJ4rwfODd6dMSe3H8YR%2BeKzworU5QgkrKRejojAMwMUkhKpvDaPW3bQ1FUN4eFL996qHfwA%3D&RelayState=ver%3A1-hint

# Use case 1: working with a single small table

In this example, we read the Snowhouse table `SAMPLE_DATA.TPCH_SF1.CUSTOMER`, which is 150k rows and 10.3 MB in Snowflake.

In [2]:
%%time

df = pd.read_snowflake("SAMPLE_DATA.TPCH_SF1.CUSTOMER")

Snapshot source table/view 'SAMPLE_DATA.TPCH_SF1.CUSTOMER' failed due to reason: `003029 (0A000): SQL compilation error:
Cannot clone from a table that was imported from a share.'. Data from source table/view 'SAMPLE_DATA.TPCH_SF1.CUSTOMER' is being copied into a new temporary table 'SNOWPARK_TEMP_TABLE_4BHELI4ZHP' for snapshotting. DataFrame creation might take some time.


CPU times: user 88.5 ms, sys: 37.9 ms, total: 126 ms
Wall time: 4.3 s


Just printing the data is visibly slow...

In [3]:
df

,C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT
0,30001,Customer#000030001,"Ui1b,3Q71CiLTJn4MbVp,,YCZARIaNTelfst",4,14-526-204-4500,8848.47,MACHINERY,frays wake blithely enticingly ironic asymptote
1,30002,Customer#000030002,UVBoMtILkQu1J3v,11,21-340-653-9800,5221.81,MACHINERY,he slyly ironic pinto beans wake slyly above t...
2,30003,Customer#000030003,CuGi9fwKn8JdR,21,31-757-493-7525,3014.89,BUILDING,e furiously alongside of the requests. evenly ...
3,30004,Customer#000030004,tkR93ReOnf9zYeO,23,33-870-136-4375,3308.55,AUTOMOBILE,ssly bold deposits. final req
4,30005,Customer#000030005,pvq4uDoD8pEwpAE01aesCtbD9WU8qmlsvoFav5,9,19-144-468-5416,-278.54,MACHINERY,ructions behind the pinto beans x-ra
...,...,...,...,...,...,...,...,...
149995,29996,Customer#000029996,BnZVGZiAgcEImNm9iD,7,17-536-308-8025,4035.17,FURNITURE,"ual instructions. bold, silent foxes nag blith..."
149996,29997,Customer#000029997,lTbDYXdQ74JctD UbRbXCqF2b8,9,19-631-777-4123,2015.90,HOUSEHOLD,eodolites detect slyly alongside of the quickl...
149997,29998,Customer#000029998,ZxxiuDruzi98CcymR,23,33-619-315-9722,-810.56,FURNITURE,xpress packages. accounts sleep carefully iron...
149998,29999,Customer#000029999,CuPA4UpgTCYiXrBrpiSO D,12,22-824-951-8333,3865.14,FURNITURE,eposits-- accounts haggle across the slyly per...


and doing certain complex transformations is very slow.

In [4]:
%%time

result = df.groupby('C_NATIONKEY').apply(lambda group: group.C_CUSTKEY.iloc[0] + group.C_CUSTKEY.mean())

KeyboardInterrupt: 

## Let's switch the backend to python!

We pay a one-time cost of a few seconds to load the data into memory.

In [5]:
%%time

df = df.move_to('Pandas')

Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 799 ms, sys: 157 ms, total: 956 ms
Wall time: 3.34 s


But now printing is extremely fast...

In [6]:
type(df.dtypes)

pandas.core.series.Series

In [7]:
df

,C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT
0,30001,Customer#000030001,"Ui1b,3Q71CiLTJn4MbVp,,YCZARIaNTelfst",4,14-526-204-4500,8848.47,MACHINERY,frays wake blithely enticingly ironic asymptote
1,30002,Customer#000030002,UVBoMtILkQu1J3v,11,21-340-653-9800,5221.81,MACHINERY,he slyly ironic pinto beans wake slyly above t...
2,30003,Customer#000030003,CuGi9fwKn8JdR,21,31-757-493-7525,3014.89,BUILDING,e furiously alongside of the requests. evenly ...
3,30004,Customer#000030004,tkR93ReOnf9zYeO,23,33-870-136-4375,3308.55,AUTOMOBILE,ssly bold deposits. final req
4,30005,Customer#000030005,pvq4uDoD8pEwpAE01aesCtbD9WU8qmlsvoFav5,9,19-144-468-5416,-278.54,MACHINERY,ructions behind the pinto beans x-ra
...,...,...,...,...,...,...,...,...
149995,29996,Customer#000029996,BnZVGZiAgcEImNm9iD,7,17-536-308-8025,4035.17,FURNITURE,"ual instructions. bold, silent foxes nag blith..."
149996,29997,Customer#000029997,lTbDYXdQ74JctD UbRbXCqF2b8,9,19-631-777-4123,2015.90,HOUSEHOLD,eodolites detect slyly alongside of the quickl...
149997,29998,Customer#000029998,ZxxiuDruzi98CcymR,23,33-619-315-9722,-810.56,FURNITURE,xpress packages. accounts sleep carefully iron...
149998,29999,Customer#000029999,CuPA4UpgTCYiXrBrpiSO D,12,22-824-951-8333,3865.14,FURNITURE,eposits-- accounts haggle across the slyly per...


And so are most things we can imagine doing with the data.

In [8]:
%%time

result = df.groupby('C_NATIONKEY').apply(lambda group: group.C_CUSTKEY.iloc[0] + group.C_CUSTKEY.mean())
print(result)

C_NATIONKEY
0     104809.648945
1     104849.461925
2     105879.897816
3     104446.823588
4     104679.314262
5     104906.427251
6     105917.388361
7     104694.312288
8     105173.743462
9     105037.669210
10    105448.956232
11    104112.225725
12    104829.006557
13    106009.666832
14    104419.615654
15    105264.778247
16    104199.408269
17    105125.565188
18    104305.006972
19    105746.410656
20    104615.581809
21    105924.703395
22    105530.506581
23    105457.153718
24    104223.723884
dtype: float64
CPU times: user 54.4 ms, sys: 7.43 ms, total: 61.8 ms
Wall time: 59.7 ms


When we're done with our transformations, we can write the results to Snowflake.

In [10]:
%%time

snow_result = result.rename('result').move_to('Snowflake')

Transferring data from Pandas to Snowflake ...:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 29.6 ms, sys: 7.31 ms, total: 36.9 ms
Wall time: 767 ms


In [11]:
%%time

snow_result.to_snowflake('TEMP.MVASHISHTHA.RESULT', if_exists='replace', index=False)

CPU times: user 20.9 ms, sys: 7.17 ms, total: 28.1 ms
Wall time: 826 ms


# Use case 2: Filtering out most of a large table, then using python

## Setup

In this example, we read the table `SAMPLE_DATA.TPCH_SF10.LINEITEM`, which is 60M rows and 1.3 GB.

In [12]:
%%time

df = pd.read_snowflake("SAMPLE_DATA.TPCH_SF10.LINEITEM")

Snapshot source table/view 'SAMPLE_DATA.TPCH_SF10.LINEITEM' failed due to reason: `003029 (0A000): SQL compilation error:
Cannot clone from a table that was imported from a share.'. Data from source table/view 'SAMPLE_DATA.TPCH_SF10.LINEITEM' is being copied into a new temporary table 'SNOWPARK_TEMP_TABLE_AQQ1QVOKYK' for snapshotting. DataFrame creation might take some time.


CPU times: user 70.4 ms, sys: 24.5 ms, total: 94.9 ms
Wall time: 10.2 s


The data is large, and pulling it all into pandas would take about 2.5 minutes...

In [18]:
#df.set_backend('Pandas')

Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

,L_ORDERKEY,L_PARTKEY,L_SUPPKEY,L_LINENUMBER,L_QUANTITY,L_EXTENDEDPRICE,L_DISCOUNT,L_TAX,L_RETURNFLAG,L_LINESTATUS,L_SHIPDATE,L_COMMITDATE,L_RECEIPTDATE,L_SHIPINSTRUCT,L_SHIPMODE,L_COMMENT
0,56224388,804093,54110,2,5.0,4985.25,0.04,0.08,N,O,1998-04-25,1998-05-30,1998-05-24,COLLECT COD,MAIL,inst the exp
1,56224388,387466,87467,3,13.0,20194.85,0.04,0.04,N,O,1998-03-29,1998-04-15,1998-04-12,DELIVER IN PERSON,REG AIR,"e regular, spe"
2,56224388,127861,27862,4,26.0,49110.36,0.09,0.02,N,O,1998-05-14,1998-05-25,1998-06-06,TAKE BACK RETURN,MAIL,ve the final ideas cajole slyly
3,56224388,955237,55238,5,38.0,49103.22,0.10,0.01,N,O,1998-03-08,1998-04-19,1998-03-25,NONE,AIR,sts sleep around the slow
4,56224389,16386,41387,1,26.0,33861.88,0.04,0.00,A,F,1995-05-17,1995-06-14,1995-05-18,COLLECT COD,MAIL,ideas. slyly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59986047,59974884,889130,39147,3,24.0,26858.16,0.03,0.06,R,F,1993-01-10,1993-02-24,1993-01-18,DELIVER IN PERSON,RAIL,the carefully express fo
59986048,59974884,1596930,71976,4,3.0,6080.58,0.08,0.08,R,F,1993-02-14,1993-02-17,1993-03-15,COLLECT COD,AIR,ove the regularly ironic
59986049,59974884,1588342,13358,5,46.0,65792.42,0.08,0.06,R,F,1993-03-21,1993-03-04,1993-04-06,DELIVER IN PERSON,TRUCK,onic requests sleep acco
59986050,59974884,1249071,24108,6,44.0,44880.44,0.09,0.04,A,F,1993-02-07,1993-02-17,1993-02-20,DELIVER IN PERSON,TRUCK,about the ideas. bold


But we only need to work with a sample of the data, so we sample 2% of the data.

In [13]:
%%time

filtered = df.sample(frac=0.02)

CPU times: user 76.4 ms, sys: 15.9 ms, total: 92.3 ms
Wall time: 4.18 s


Now it's easier to fetch the data.

In [15]:
%%time

python_filtered = filtered.move_to('PANDAS')

Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 2.36 s, sys: 396 ms, total: 2.75 s
Wall time: 4.93 s


Now we can do a complex operation on the filtered subset of the data.

In [16]:
%%time

python_filtered.groupby('L_SHIPMODE').apply(lambda group: group.L_ORDERKEY.iloc[0] + group.L_ORDERKEY.mean())

CPU times: user 254 ms, sys: 31.8 ms, total: 285 ms
Wall time: 283 ms


L_SHIPMODE
AIR        4.314149e+07
FOB        4.319355e+07
MAIL       4.317048e+07
RAIL       4.314031e+07
REG AIR    4.316588e+07
SHIP       4.321064e+07
TRUCK      4.317835e+07
dtype: float64

Doing the same operation on the filtered data in Snowflake would take much longer.

In [17]:
%%time

filtered.groupby('L_SHIPMODE').apply(lambda group: group.L_ORDERKEY.iloc[0] + group.L_ORDERKEY.mean())

KeyboardInterrupt: 

In [27]:
# Use case 3: Intermingling engines

In [2]:
snow_df = pd.read_snowflake("SAMPLE_DATA.TPCH_SF10.LINEITEM")

Snapshot source table/view 'SAMPLE_DATA.TPCH_SF10.LINEITEM' failed due to reason: `003029 (0A000): SQL compilation error:
Cannot clone from a table that was imported from a share.'. Data from source table/view 'SAMPLE_DATA.TPCH_SF10.LINEITEM' is being copied into a new temporary table 'SNOWPARK_TEMP_TABLE_2YYUP5L6KH' for snapshotting. DataFrame creation might take some time.


In [3]:
%%time

native_df = snow_df['L_SHIPMODE'].value_counts().move_to('Pandas')
native_df

Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 212 ms, sys: 84.8 ms, total: 297 ms
Wall time: 1.5 s


L_SHIPMODE
RAIL       8571844
SHIP       8571402
REG AIR    8570280
FOB        8569760
MAIL       8569053
TRUCK      8567549
AIR        8566164
Name: count, dtype: int64

In [7]:
snow_df.columns

Index(['L_ORDERKEY', 'L_PARTKEY', 'L_SUPPKEY', 'L_LINENUMBER', 'L_QUANTITY',
       'L_EXTENDEDPRICE', 'L_DISCOUNT', 'L_TAX', 'L_RETURNFLAG',
       'L_LINESTATUS', 'L_SHIPDATE', 'L_COMMITDATE', 'L_RECEIPTDATE',
       'L_SHIPINSTRUCT', 'L_SHIPMODE', 'L_COMMENT'],
      dtype='object')

In [12]:
snow_df.get_backend()

'Snowflake'

In [9]:
native_df

L_SHIPMODE
RAIL       8571844
SHIP       8571402
REG AIR    8570280
FOB        8569760
MAIL       8569053
TRUCK      8567549
AIR        8566164
Name: count, dtype: int64

In [10]:
snow_df.merge(native_df, on="L_SHIPMODE")

Transferring data from Pandas to Snowflake ...:   0%|          | 0/2 [00:00<?, ?it/s]

,L_ORDERKEY,L_PARTKEY,L_SUPPKEY,L_LINENUMBER,L_QUANTITY,L_EXTENDEDPRICE,L_DISCOUNT,L_TAX,L_RETURNFLAG,L_LINESTATUS,L_SHIPDATE,L_COMMITDATE,L_RECEIPTDATE,L_SHIPINSTRUCT,L_SHIPMODE,L_COMMENT,count
0,27517350,1896835,71890,5,42.0,76933.08,0.08,0.08,R,F,1992-10-31,1992-10-17,1992-11-30,DELIVER IN PERSON,FOB,oxes unwind,8569760
1,27517351,665715,65716,1,22.0,36974.96,0.08,0.08,N,O,1997-03-15,1997-01-13,1997-03-18,COLLECT COD,AIR,e across the bli,8566164
2,27517351,666667,16680,2,39.0,63711.57,0.09,0.03,N,O,1997-04-11,1997-01-22,1997-04-24,COLLECT COD,TRUCK,beans sleep care,8567549
3,27517351,671029,71030,3,2.0,1999.98,0.03,0.06,N,O,1997-02-27,1997-02-18,1997-03-12,DELIVER IN PERSON,SHIP,ual theodolites. careful,8571402
4,27517351,1977061,2081,4,2.0,2275.94,0.07,0.01,N,O,1997-03-13,1997-02-21,1997-04-06,NONE,RAIL,uickly along th,8571844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59986047,19721668,1258271,33308,1,15.0,18438.15,0.01,0.04,N,O,1995-09-02,1995-07-14,1995-09-16,DELIVER IN PERSON,SHIP,l foxes detect carefully. slyly r,8571402
59986048,19721668,1753262,3297,2,38.0,49976.84,0.02,0.00,N,O,1995-07-17,1995-08-26,1995-08-15,NONE,RAIL,ly final pa,8571844
59986049,19721669,124306,24307,1,28.0,37248.40,0.02,0.08,A,F,1992-09-30,1992-09-21,1992-10-01,TAKE BACK RETURN,MAIL,ole carefully slyly final deposits. ca,8569053
59986050,19721669,1194302,44325,2,36.0,50265.00,0.03,0.04,R,F,1992-10-24,1992-08-26,1992-10-25,TAKE BACK RETURN,TRUCK,theodolites,8567549


In [8]:
snow_df.dtypes

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://snowbiz.okta.com/app/snowflake/exk8wfsfryJIn4IWZ2p7/sso/saml?SAMLRequest=jVJdc9owEPwrHvXZlgykIRogQ0MozpBAMJApb8KWQUGWXJ0cQ399ZT4y6UMyfdOcdm%2F3bq9zu8%2Bl98YNCK26KAwI8rhKdCrUposW86HfRh5YplImteJddOCAbnsdYLksaL%2B0WzXjv0sO1nONFND6o4tKo6hmIIAqlnOgNqFx%2F3FMGwGhDIAb6%2BTQmZKCcFpbawuKcVVVQdUMtNngBiEEkxvsUDXkG%2FogUXytURhtdaLlhbJ3M30iEWLSqiUcwilMz8QfQp1W8JXK%2BgQCOprPp%2F50Es%2BR179Md6cVlDk3MTdvIuGL2fhkAJyD%2BGnyMpos4vsAlK4yyXY80XlRWtctcC%2Bc8RRLvRFuR9Ggi4qdSAd2uFTL3LD16iEO5fS%2BPRPZ6%2Ffxq9iS9fO%2Btbnu%2F1yLKh4dfiXIW14SbdSJRgAlj1Sdo3Ul0rjySdNvhvPwhl6FNAyDNiEr5A1cjkIxe2RezNYW1%2BJPoHeWHc2xosDvvjHf79pVBpk5PESqFb2sGsU1BtC4jgmdLoUeDZje%2F87fwR9Z52N7cvuPBlMtRXLwhtrkzH4eTxiEx4pI%2FewIpTxnQvbT1HAAF5OUuroznFl309aUHOHeSfXfq%2B79BQ%3D%3D&RelayState=

SnowparkSQLException: (1304): 01bb61e7-080f-702e-0001-dd47a26ef3df: 002003 (42S02): SQL compilation error:
Object 'SNOWPARK_TEMP_TABLE_AVZFAMGLDDREADONLY' does not exist or not authorized.

In [9]:
native_df.dtypes

dtype('int64')

# Move on read

In [1]:
%%time

import snowflake.snowpark.modin.plugin
import modin.pandas as pd
import numpy as np
import datetime
import pandas as native_pd
from snowflake.snowpark.session import Session; session = Session.builder.create()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://snowbiz.okta.com/app/snowflake/exk8wfsfryJIn4IWZ2p7/sso/saml?SAMLRequest=jVJdc9owEPwrHvUZyxbQBA0m40JTnAkfwRCmeVNsAQq2ZHQyhv76ynxk0odk%2BqY57d7u3V737pBnzp5rEEoGyHc95HCZqFTIdYAW8%2FvGLXLAMJmyTEkeoCMHdNfrAsuzgoal2cgZ35UcjGMbSaD1R4BKLaliIIBKlnOgJqFxOHqkxPUoA%2BDaWDl0oaQgrNbGmIJiXFWVWzVdpdeYeJ6HvQ62qBryDX2QKL7WKLQyKlHZlXKwM30i4WOvVUtYhFWYXog%2FhDyv4CuV1zMI6HA%2Bnzamk3iOnPA6XV9JKHOuY673IuGL2ePZAFgH8XiyHE4W8U8XpKpWGdvyROVFaWw3177wiqc4U2thdxQNAlRsRfq2I099Mp60Z9vw96Q52MSkGZb7X7tEL0ftB%2FacDpOjiePRW5Ug5%2FmaKKkTjQBKHsk6R2NLHmk3vFbD8%2BeE0HaHko7b%2Fn7zgpyBzVFIZk7Mq9na4qv446qtYSdzrCjwu2%2FMD9vbagUrfXyIZCtavpDiBgMoXMeEzpdCTwZ073%2Fn7%2BKPrMuxje3%2Bo8FUZSI5OvdK58x8Ho%2Fv%2BqeKSBurE5TynIksTFPNAWxMWaaqvubM2Js2uuQI986q%2F1517y8%3D&RelayState=ver%3A1-

In [2]:
df = pd.read_snowflake("SAMPLE_DATA.TPCH_SF1.CUSTOMER")

Snapshot source table/view 'SAMPLE_DATA.TPCH_SF1.CUSTOMER' failed due to reason: `003029 (0A000): SQL compilation error:
Cannot clone from a table that was imported from a share.'. Data from source table/view 'SAMPLE_DATA.TPCH_SF1.CUSTOMER' is being copied into a new temporary table 'SNOWPARK_TEMP_TABLE_UHHXNNNHF8' for snapshotting. DataFrame creation might take some time.


Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
df.get_backend()

'Pandas'

In [4]:
df

,C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT
0,30001,Customer#000030001,"Ui1b,3Q71CiLTJn4MbVp,,YCZARIaNTelfst",4,14-526-204-4500,8848.47,MACHINERY,frays wake blithely enticingly ironic asymptote
1,30002,Customer#000030002,UVBoMtILkQu1J3v,11,21-340-653-9800,5221.81,MACHINERY,he slyly ironic pinto beans wake slyly above t...
2,30003,Customer#000030003,CuGi9fwKn8JdR,21,31-757-493-7525,3014.89,BUILDING,e furiously alongside of the requests. evenly ...
3,30004,Customer#000030004,tkR93ReOnf9zYeO,23,33-870-136-4375,3308.55,AUTOMOBILE,ssly bold deposits. final req
4,30005,Customer#000030005,pvq4uDoD8pEwpAE01aesCtbD9WU8qmlsvoFav5,9,19-144-468-5416,-278.54,MACHINERY,ructions behind the pinto beans x-ra
...,...,...,...,...,...,...,...,...
149995,29996,Customer#000029996,BnZVGZiAgcEImNm9iD,7,17-536-308-8025,4035.17,FURNITURE,"ual instructions. bold, silent foxes nag blith..."
149996,29997,Customer#000029997,lTbDYXdQ74JctD UbRbXCqF2b8,9,19-631-777-4123,2015.90,HOUSEHOLD,eodolites detect slyly alongside of the quickl...
149997,29998,Customer#000029998,ZxxiuDruzi98CcymR,23,33-619-315-9722,-810.56,FURNITURE,xpress packages. accounts sleep carefully iron...
149998,29999,Customer#000029999,CuPA4UpgTCYiXrBrpiSO D,12,22-824-951-8333,3865.14,FURNITURE,eposits-- accounts haggle across the slyly per...


In [5]:
df_big = pd.read_snowflake("SAMPLE_DATA.TPCH_SF10.CUSTOMER")

Snapshot source table/view 'SAMPLE_DATA.TPCH_SF10.CUSTOMER' failed due to reason: `003029 (0A000): SQL compilation error:
Cannot clone from a table that was imported from a share.'. Data from source table/view 'SAMPLE_DATA.TPCH_SF10.CUSTOMER' is being copied into a new temporary table 'SNOWPARK_TEMP_TABLE_BCG1A32YWB' for snapshotting. DataFrame creation might take some time.


In [6]:
df_big.get_backend()

'Snowflake'